# 연습문제 해답

## 12. 조기 종료를 사용한 배치 경사 하강법으로 소프트맥스 회귀 구현하기
문제: _사이킷런을 사용하지 않고 넘파이만으로 조기 종료를 사용한 배치 경사 하강법으로 소프트맥스 회귀를 구현해 보세요. 이를 붓꽃 데이터셋과 같은 분류 작업에 사용해 보세요._

데이터를 로드하는 것부터 시작하겠습니다. 앞서 로드한 붓꽃 데이터셋을 재사용하겠습니다.

In [62]:
X = iris.data[["petal length (cm)", "petal width (cm)"]].values
y = iris["target"].values

모든 샘플에 대해 편향 항을 추가해야 합니다($x_0 = 1$). 이 작업을 수행하는 가장 쉬운 옵션은 사이킷런의 `add_dummy_feature()` 함수를 사용하는 것이지만, 이 연습의 요점은 알고리즘을 수동으로 구현하여 알고리즘을 더 잘 이해하도록 하는 것입니다. 그래서 한 가지 가능한 구현은 다음과 같습니다:

In [63]:
X_with_bias = np.c_[np.ones(len(X)), X]

데이터셋을 훈련 세트, 검증 세트 및 테스트 세트로 분할하는 가장 쉬운 옵션은 사이킷런의 `train_test_split()` 함수를 사용하는 것이지만, 이 역시 수동으로 수행합니다:

In [64]:
test_ratio = 0.2
validation_ratio = 0.2
total_size = len(X_with_bias)

test_size = int(total_size * test_ratio)
validation_size = int(total_size * validation_ratio)
train_size = total_size - test_size - validation_size

np.random.seed(42)
rnd_indices = np.random.permutation(total_size)

X_train = X_with_bias[rnd_indices[:train_size]]
y_train = y[rnd_indices[:train_size]]
X_valid = X_with_bias[rnd_indices[train_size:-test_size]]
y_valid = y[rnd_indices[train_size:-test_size]]
X_test = X_with_bias[rnd_indices[-test_size:]]
y_test = y[rnd_indices[-test_size:]]

현재 타깃은 클래스 인덱스(0, 1 또는 2)이지만, 소프트맥스 회귀 모델을 훈련하려면 타깃 클래스 확률이 필요합니다. 각 샘플은 확률이 1.0인 타깃 클래스를 제외한 모든 클래스에 대해 0.0인 타깃 클래스 확률을 갖습니다(즉, 주어진 샘플에 대한 클래스 확률 벡터는 원-핫 벡터입니다). 클래스 인덱스 벡터를 각 샘플에 대한 원-핫 벡터를 포함하는 행렬로 변환하는 작은 함수를 작성해 보겠습니다. 이 코드를 이해하려면 `np.diag(np.ones(n))`가 주 대각선의 1을 제외한 0으로 가득 찬 n×n 행렬을 생성한다는 사실을 알아야 합니다. 또한 `a`가 넘파이 배열인 경우 `a[[1, 3, 2]]`는 `a[1]`, `a[3]` 및 `a[2]`와 동일한 3개의 행을 가진 배열을 반환합니다(이것이 [고급 넘파이 인덱싱](https://numpy.org/doc/stable/user/basics.indexing.html#advanced-indexing)입니다).

In [65]:
def to_one_hot(y):
    return np.diag(np.ones(y.max() + 1))[y]

처음 10개의 샘플에서 이 함수를 테스트해 보겠습니다:

In [ ]:
y_train[:10]

In [ ]:
to_one_hot(y_train[:10])

이제 훈련 세트와 테스트 세트에 대한 타깃 클래스 확률 행렬을 생성해 보겠습니다:

In [68]:
Y_train_one_hot = to_one_hot(y_train)
Y_valid_one_hot = to_one_hot(y_valid)
Y_test_one_hot = to_one_hot(y_test)

이제 입력의 스케일을 조정해 보겠습니다. 훈련 세트에서 (편향은 제외한) 각 특성의 평균과 표준 편차를 계산한 다음 훈련 세트, 검증 세트 및 테스트 세트에서 각 특성을 중앙에 맞추고 스케일을 조정합니다:

In [69]:
mean = X_train[:, 1:].mean(axis=0)
std = X_train[:, 1:].std(axis=0)
X_train[:, 1:] = (X_train[:, 1:] - mean) / std
X_valid[:, 1:] = (X_valid[:, 1:] - mean) / std
X_test[:, 1:] = (X_test[:, 1:] - mean) / std

이제 소프트맥스 함수를 구현해 보겠습니다. 다음 방정식으로 정의된다는 것을 기억하세요:

$\sigma\left(\mathbf{s}(\mathbf{x})\right)_k = \dfrac{\exp\left(s_k(\mathbf{x})\right)}{\sum\limits_{j=1}^{K}{\exp\left(s_j(\mathbf{x})\right)}}$

In [70]:
def softmax(logits):
    exps = np.exp(logits)
    exp_sums = exps.sum(axis=1, keepdims=True)
    return exps / exp_sums

훈련을 시작할 준비가 거의 다 되었습니다. 입력과 출력의 수를 정의해 보겠습니다:

In [71]:
n_inputs = X_train.shape[1]  # == 3 (2개의 특성과 편향)
n_outputs = len(np.unique(y_train))  # == 3 (3개의 붓꽃 클래스)

이제 가장 어려운 부분인 훈련입니다! 이론적으로는 간단합니다. 수학 방정식을 파이썬 코드로 변환하기만 하면 됩니다. 하지만 실제로는 상당히 까다로울 수 있습니다. 특히 항의 순서나 인덱스가 혼동되기 쉽습니다. 심지어 제대로 작동하는 것처럼 보이지만 실제로는 정확히 계산되지 않는 코드가 나올 수도 있습니다. 확실하지 않은 경우 방정식의 각 항의 크기를 적어두고 코드의 해당 항이 정확히 일치하는지 확인해야 합니다. 각 항을 독립적으로 평가하고 인쇄하는 것도 도움이 될 수 있습니다. 좋은 소식은 이 모든 것이 사이킷런에 의해 잘 구현되어 있기 때문에 매일 이 작업을 수행할 필요는 없지만, 내부에서 무슨 일이 일어나고 있는지 이해하는 데 도움이 된다는 것입니다.

따라서 우리에게 필요한 방정식은 비용 함수입니다:

$J(\mathbf{\Theta}) =
- \dfrac{1}{m}\sum\limits_{i=1}^{m}\sum\limits_{k=1}^{K}{y_k^{(i)}\log\left(\hat{p}_k^{(i)}\right)}$

그리고 그레이디언트 방정식입니다:

$\nabla_{\mathbf{\theta}^{(k)}} \, J(\mathbf{\Theta}) = \dfrac{1}{m} \sum\limits_{i=1}^{m}{ \left ( \hat{p}^{(i)}_k - y_k^{(i)} \right ) \mathbf{x}^{(i)}}$

$\hat{p}_k^{(i)} = 0$일 때 $\log\left(\hat{p}_k^{(i)}\right)$는 계산할 수 없습니다. 따라서 $\log\left(\hat{p}_k^{(i)}\right)$에 작은 값 $\epsilon$을 더해 `nan` 값을 피합니다.

In [ ]:
eta = 0.5
n_epochs = 5001
m = len(X_train)
epsilon = 1e-5

np.random.seed(42)
Theta = np.random.randn(n_inputs, n_outputs)

for epoch in range(n_epochs):
    logits = X_train @ Theta
    Y_proba = softmax(logits)
    if epoch % 1000 == 0:
        Y_proba_valid = softmax(X_valid @ Theta)
        xentropy_losses = -(Y_valid_one_hot * np.log(Y_proba_valid + epsilon))
        print(epoch, xentropy_losses.sum(axis=1).mean())
    error = Y_proba - Y_train_one_hot
    gradients = 1 / m * X_train.T @ error
    Theta = Theta - eta * gradients

이게 다입니다! 소프트맥스 모델이 학습되었습니다. 모델 파라미터를 살펴봅시다:

In [ ]:
Theta

검증 세트에 대한 예측을 수행하고 정확도 점수를 확인해 보겠습니다:

In [ ]:
logits = X_valid @ Theta
Y_proba = softmax(logits)
y_predict = Y_proba.argmax(axis=1)

accuracy_score = (y_predict == y_valid).mean()
accuracy_score

이 모델은 꽤 괜찮아 보입니다. 연습을 위해 약간의 $\ell_2$ 정규화를 추가해 보겠습니다. 다음 훈련 코드는 위의 코드와 유사하지만 손실에 $\ell_2$ 페널티가 추가되고 그레이디언트가 적절한 추가 항을 가집니다(`Theta`의 첫 번째 원소는 편향에 해당하므로 규제하지 않음). 또한 학습률 `eta`를 높여 보겠습니다.

In [ ]:
eta = 0.5
n_epochs = 5001
m = len(X_train)
epsilon = 1e-5
alpha = 0.01  # 규제 하이퍼파라미터

np.random.seed(42)
Theta = np.random.randn(n_inputs, n_outputs)

for epoch in range(n_epochs):
    logits = X_train @ Theta
    Y_proba = softmax(logits)
    if epoch % 1000 == 0:
        Y_proba_valid = softmax(X_valid @ Theta)
        xentropy_losses = -(Y_valid_one_hot * np.log(Y_proba_valid + epsilon))
        l2_loss = 1 / 2 * (Theta[1:] ** 2).sum()
        total_loss = xentropy_losses.sum(axis=1).mean() + alpha * l2_loss
        print(epoch, total_loss.round(4))
    error = Y_proba - Y_train_one_hot
    gradients = 1 / m * X_train.T @ error
    gradients += np.r_[np.zeros([1, n_outputs]), alpha * Theta[1:]]
    Theta = Theta - eta * gradients

추가 $\ell_2$ 페널티로 인해 손실이 이전보다 더 커 보이지만 이 모델이 더 나은 성과를 낼 수 있을까요? 알아봅시다:

In [ ]:
logits = X_valid @ Theta
Y_proba = softmax(logits)
y_predict = Y_proba.argmax(axis=1)

accuracy_score = (y_predict == y_valid).mean()
accuracy_score

이 경우 $\ell_2$ 페널티로 인해 테스트 정확도가 변경되지 않았습니다. `alpha`를 미세 튜닝해 볼 수 있을까요?

이제 조기 종료를 추가해 보겠습니다. 이를 위해 모든 반복에서 검증 세트의 손실을 측정하고 오차가 증가하기 시작하면 중지하기만 하면 됩니다.

In [ ]:
eta = 0.5
n_epochs = 50_001
m = len(X_train)
epsilon = 1e-5
C = 100  # 규제 하이퍼파라미터
best_loss = np.infty

np.random.seed(42)
Theta = np.random.randn(n_inputs, n_outputs)

for epoch in range(n_epochs):
    logits = X_train @ Theta
    Y_proba = softmax(logits)
    Y_proba_valid = softmax(X_valid @ Theta)
    xentropy_losses = -(Y_valid_one_hot * np.log(Y_proba_valid + epsilon))
    l2_loss = 1 / 2 * (Theta[1:] ** 2).sum()
    total_loss = xentropy_losses.sum(axis=1).mean() + 1 / C * l2_loss
    if epoch % 1000 == 0:
        print(epoch, total_loss.round(4))
    if total_loss < best_loss:
        best_loss = total_loss
    else:
        print(epoch - 1, best_loss.round(4))
        print(epoch, total_loss.round(4), "조기 종료!")
        break
    error = Y_proba - Y_train_one_hot
    gradients = 1 / m * X_train.T @ error
    gradients += np.r_[np.zeros([1, n_outputs]), 1 / C * Theta[1:]]
    Theta = Theta - eta * gradients

In [ ]:
logits = X_valid @ Theta
Y_proba = softmax(logits)
y_predict = Y_proba.argmax(axis=1)

accuracy_score = (y_predict == y_valid).mean()
accuracy_score

여전히 검증 정확도에는 변화가 없지만, 적어도 조기 중지로 인해 훈련 시간이 조금 단축되었습니다.

이제 전체 데이터셋에 대한 모델의 예측을 그래프로 그려 보겠습니다(모델에 주입하는 모든 특성의 크기를 조정하는 것을 잊지 마세요):

In [ ]:
custom_cmap = mpl.colors.ListedColormap(['#fafab0', '#9898ff', '#a0faa0'])

x0, x1 = np.meshgrid(np.linspace(0, 8, 500).reshape(-1, 1),
                     np.linspace(0, 3.5, 200).reshape(-1, 1))
X_new = np.c_[x0.ravel(), x1.ravel()]
X_new = (X_new - mean) / std
X_new_with_bias = np.c_[np.ones(len(X_new)), X_new]

logits = X_new_with_bias @ Theta
Y_proba = softmax(logits)
y_predict = Y_proba.argmax(axis=1)

zz1 = Y_proba[:, 1].reshape(x0.shape)
zz = y_predict.reshape(x0.shape)

plt.figure(figsize=(10, 4))
plt.plot(X[y == 2, 0], X[y == 2, 1], "g^", label="Iris virginica")
plt.plot(X[y == 1, 0], X[y == 1, 1], "bs", label="Iris versicolor")
plt.plot(X[y == 0, 0], X[y == 0, 1], "yo", label="Iris setosa")

plt.contourf(x0, x1, zz, cmap=custom_cmap)
contour = plt.contour(x0, x1, zz1, cmap="hot")
plt.clabel(contour, inline=1)
plt.xlabel("Petal length")
plt.ylabel("Petal width")
plt.legend(loc="upper left")
plt.axis([0, 7, 0, 3.5])
plt.grid()
plt.show()

이제 테스트 세트에서 최종 모델의 정확도를 측정해 보겠습니다:

In [ ]:
logits = X_test @ Theta
Y_proba = softmax(logits)
y_predict = Y_proba.argmax(axis=1)

accuracy_score = (y_predict == y_test).mean()
accuracy_score

테스트 세트에서 더 나은 성능을 얻었습니다. 이러한 변동성은 데이터셋의 크기가 매우 작기 때문일 수 있습니다. 훈련 세트, 검증 세트 및 테스트 세트의 샘플링 방법에 따라 상당히 다른 결과를 얻을 수 있습니다. 랜덤 시드를 변경하고 코드를 몇 번 다시 실행해 보면 결과가 달라지는 것을 확인할 수 있습니다.